In [1]:
import pandas as pd
import numpy as np

In [3]:
filename = 'data_2014'
df = pd.read_csv(f'{filename}_with_topics_full.csv')
wiki_df = pd.read_csv(f'{filename}_with_context.csv')

In [4]:
# merge the two dataframes using index
df = df.merge(wiki_df, left_index=True, right_index=True)

In [5]:
df = df[['comments', 'context']]

In [6]:
# drop rows with context as empty string
df = df[df['context'] != '']
df = df[df['context'] != 'nan']

In [7]:
import ast
df['comments'] = df['comments'].apply(ast.literal_eval)

In [8]:
# filter rows that does not have topic in the comments
z = lambda x: 'topic' in x['comments'][0]

df = df[df.apply(z, axis=1)]

In [9]:
# pivot the dataframe to have one comment per row
df_new = df.explode('comments')

In [10]:
df_new['context_index'] = df_new.index
# drop rows with context as NaN
df_new = df_new.dropna(subset=['context'])

In [11]:
# drop rows that have comments with no topic
df_new = df_new[df_new['comments'].apply(lambda x: 'topic' in x)]

In [13]:
df_new['topic'] = df_new['comments'].apply(lambda x: x['topic'])

In [16]:
df_new['text'] = df_new.apply(lambda x: f'<topic> {x["topic"]} <context> {x["context"]}', axis=1)
df_new['question'] = df_new['comments'].apply(lambda x: x['body'])

In [18]:
# df_new[['question', 'text']].to_csv('question_generator.csv', index=False)
question_generation_test = df_new[['question', 'text']].copy()

# reset the index
question_generation_test.reset_index(drop=True, inplace=True)

# shuffle the dataframe
question_generation_test = question_generation_test.sample(frac=1).reset_index(drop=True)

# save
question_generation_test.to_csv('question_generator_test.csv', index=False)
